<a href="https://colab.research.google.com/github/delenarias/practica-mongo-neo4j/blob/main/scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importar base de datos**

a-Creación de base de datos **inspection** e importar ***Collection***. El script se ejectuta en consola.

In [ ]:
mongoimport --db inspections --collection grupal_city_inspections --type json --file act-grupal-city_inspections.json

b-Creación de base de datos **countries** e importar dos ***Collection***. El script se ejecuta en consola.

**Exportar base de datos**

a-Exportar base de datos **inspection** en formato json. El script se ejecuta en consola

In [ ]:
mongoexport --collection inspections --db inspections --out C:\PROYECTOS\inspections.json

**Restaurar base de datos**

Para estos efectos colocar en la carpeta dump los archivos a restaurar y desde consola aplicar el siguiente script.

In [ ]:
mongorestore dump/

**Caso de uso: restricción de locales y terrazas en Madrid por Covid-19**

h. Alternativa para insert masivo

In [ ]:
db.locales.insert(datos_insertar)

6.2 **Actualizaciones**

a-Los locales del barrio Guindalera de Salamanca, por motivos de la desescalada no podrán abrir y tendrán que permanecer cerrados. Cambiad la situación del local a cerrado y utilizad el ID correspondiente

In [ ]:
db.locales.update({desc_barrio_local:'GUINDALERA', desc_distrito_local:'SALAMANCA'},{$set:{desc_situacion_local:'Cerrado'}},{multi:true})



•	A estos mismos locales, cambiad la situación de la terraza a cerrada siguiendo las mismas premisas anteriores.

In [ ]:
db.locales.update({desc_barrio_local:'GUINDALERA', desc_distrito_local:'SALAMANCA'},{$set:{desc_situacion_terraza:'Cerrada',id_situacion_terraza:3}},{multi:true})


b-A todas las terrazas que se ubiquen en la acera, añadid un campo llamado inspeccionar y estableced el siguiente valor:
•	Si dispone de más de 10 mesas, true.
•	Si dispone de menos de 10 mesas, false.


In [ ]:
db.locales.updateMany(
   {desc_ubicacion_terraza:'Acera'},
   [
     { $set: { inspeccionar: { $switch: {
                           branches: [
                               { case: { $gte: [ "$mesas_es", 10 ] }, then: true },
                               { case: { $lt: [ "$mesas_es", 10 ] }, then: false} 
                           ],
                           default: ""
     } } } }
   ]
)

c-A las terrazas que se deban inspeccionar, asignad 2 mesas auxiliares y 8 sillas disponibles. 

In [ ]:
db.locales.update({inspeccionar:true},{$set:{mesas_aux_es:2,sillas_ra:8}},{multi:true})


d- A las terrazas que no deban ser inspeccionadas, añadid el campo estado con el valor:
•	1 si el número de sillas es menor que 10.
•	2 si el número de sillas está entre 10 y 20.
•	3 si cuenta con más de 20 sillas.


In [ ]:
db.locales.updateMany(
   {inspeccionar:false},
   [
     { $set: { estado: { $switch: {
                           branches: [
                               { case: { $lt: [ "$sillas_es", 10 ] }, then: 1 },
                               { case: { $and :[{$gte: [ "$sillas_es", 10 ]},{ $lt: [ "$sillas_es", 20 ]}]}, then: 2},
                               { case: { $gte: [ "$sillas_es", 20 ] }, then: 3}
                           ],
                           default: ""
     } } } }
   ]
)

e-De lunes a jueves, ningún local podrá cerrar más allá de las 00:00:00, actualizad el horario de cierre a esta nueva hora límite. 

In [ ]:
db.locales.update({hora_fin_LJ_es:{$gt:'0:00:00'}},{$set:{hora_fin_LJ_es:'0:00:00'}},{multi:true})


f-De viernes a sábado, los locales que cierren a las 2:30:00 ahora tendrán que hacerlo a las 2:00:00

In [ ]:
db.locales.update({hora_fin_VS_es:{$eq:'2:30:00'}},{$set:{hora_fin_VS_es:'2:00:00'}},{multi:true})


g-A todos los locales que estén sobre la calle Alcalá en Madrid se les debe inspeccionar. 

In [ ]:
db.locales.update({DESC_CLASE:'CALLE',DESC_NOMBRE:'ALCALA'},{$set:{inspeccionar:true}},{multi:true})


h-A todos los locales con terrazas abiertas, añadid un campo revision cuyo valor sea un documento con la siguiente información: {prox_inspeccion: 10, puntuacion: 80, comentario: “separar las mesas”}

In [ ]:
db.locales.update({desc_situacion_terraza:'Abierta'},{$set:{revision:{prox_inspeccion: 10, puntuacion: 80, comentario: 'separar las mesas'}}},{multi:true})


i-Cread una nueva colección llamada zona1 con todos los locales que pertenezcan al distrito de Villaverde.

In [ ]:
cur = db.locales.find({desc_distrito_local:'VILLAVERDE'});
while (cur.hasNext()) { db.zona_1.insert(cur.next()) }

db.zona_1.find()

j-Cread una nueva colección llamada zona2 con todos los locales que pertenezcan al distrito de Salamanca y barrio Castellana.

In [ ]:
cur = db.locales.find({desc_distrito_local:'SALAMANCA', desc_barrio_local:'CASTELLANA'});
while (cur.hasNext()) { db.zona_2.insert(cur.next()) }

db.zona_2.find()